## Setup

In [1]:
%cd ..

d:\Blockchain\data\orai\orchai_validator


In [2]:
from omegaconf import OmegaConf
from labeling.tools import psql_connect
import numpy as np
from labeling.upload import get_batch_intervals

In [3]:
config = OmegaConf.load("config/etl.yaml")
cur = psql_connect(**config.dest).cursor()

vote_proposed_win_size = config.hp.etl.vote_proposed_win_size
combine_win_size = config.hp.etl.combine_win_size
label_win_size = config.hp.etl.label_win_size
batch_size = config.hp.upload.batch_size

def num_labeled_blocks(batch_size):
    return (batch_size - vote_proposed_win_size + 1) / combine_win_size - (label_win_size / combine_win_size)

## Validate

### Detect wrong block difference

In [4]:
cur.execute(f"select distinct block_height from {config.dest.table} order by block_height;")
bh = cur.fetchall()
bh = np.array([b[0] for b in bh])

In [5]:
bh.shape

(11127,)

In [6]:
from collections import Counter

Counter(np.diff(bh))

Counter({150: 11114, 300: 7, 390: 2, 60: 2, 171: 1})

### Detail wrong differences

In [16]:
diff = np.diff(bh)
commands = []

for i, d in enumerate(diff):
    if d != combine_win_size:
        if d % combine_win_size == 0:
            rs = bh[i] + combine_win_size
            re = bh[i + 1] - combine_win_size
            cmd = f"python ./uploader.py -cf ./config/etl.yaml -s {rs} -e {re + combine_win_size + label_win_size - 1}"
            commands.append(cmd)
       
        print(bh[i], bh[i+1], "|", d)
        
        bs = bh[i] - vote_proposed_win_size + 1 + combine_win_size
        be = bh[i+1] + combine_win_size + label_win_size - 1
        # print(bs, be, "|", d)

7167971 7168271 | 300
7202771 7203071 | 300
7237571 7237871 | 300
7272371 7272671 | 300
7272671 7273061 | 390
7307411 7307471 | 60
7794371 7794671 | 300
7829171 7829471 | 300
7863971 7864271 | 300
7883921 7884311 | 390
7899011 7899071 | 60
8699921 8700092 | 171


#### Commands to fill missing

In [17]:
for c in commands:
    print(f"!{c}")

!python ./uploader.py -cf ./config/etl.yaml -s 7168121 -e 7269070
!python ./uploader.py -cf ./config/etl.yaml -s 7202921 -e 7303870
!python ./uploader.py -cf ./config/etl.yaml -s 7237721 -e 7338670
!python ./uploader.py -cf ./config/etl.yaml -s 7272521 -e 7373470
!python ./uploader.py -cf ./config/etl.yaml -s 7794521 -e 7895470
!python ./uploader.py -cf ./config/etl.yaml -s 7829321 -e 7930270
!python ./uploader.py -cf ./config/etl.yaml -s 7864121 -e 7965070


In [11]:
ts = 7168121 - vote_proposed_win_size + 1
te = 7168121 + label_win_size + combine_win_size - 1
print(ts, te, te - ts + 1)
print(num_labeled_blocks(te - ts + 1))

7153722 7269070 115349
1.0


## Generate block intervals

In [9]:
intervals = get_batch_intervals(
    start_block=7063871,
    global_end_block=9291903,
    batch_size=batch_size,
    vote_proposed_win_size=vote_proposed_win_size,
    combine_win_size=combine_win_size,
    label_win_size=label_win_size,
    min_block=7049472
)

len(intervals)

62

### Split into batches

In [27]:
b_size = 3
start = None

for idx, (bs, be) in enumerate(intervals):
    if start is None:
        start = bs
    if (idx + 1) % b_size == 0:
        print(start + vote_proposed_win_size - 1, be)
        start = None

7063871 7269070
7168271 7373470
7272671 7477870
7377071 7582270
7481471 7686670
7585871 7791070
7690271 7895470
7794671 7999870
7899071 8104270
8003471 8208670
8107871 8313070
8212271 8417470
8316671 8521870
8421071 8626270
8525471 8730670
8629871 8835070
8734271 8939470
8838671 9043870
8943071 9148270
9047471 9252670


### Post-test

In [36]:
blocks = []

for bs, be in get_batch_intervals(
    start_block=7063871,
    global_end_block=7269070,
    batch_size=batch_size,
    vote_proposed_win_size=vote_proposed_win_size,
    combine_win_size=combine_win_size,
    label_win_size=label_win_size,
    min_block=7049472
):
    print(bs, be, be - bs + 1)
    blocks += list(range(bs + vote_proposed_win_size - 1, be - label_win_size - combine_win_size + 1 + 1, combine_win_size))

7049472 7199470 149999
7084272 7234270 149999
7119072 7269070 149999


In [38]:
len(blocks) / 232

3.0

In [39]:
blocks = np.array(blocks)
Counter(np.diff(blocks))

Counter({150: 695})

## Missing block:
```
7272731 7272972 | 241
7883969 7884210 | 241
8699979 8700001 | 22
9057005 9057016 | 11
9057065 9057076 | 11
```

## Total block

In [5]:
cur.execute(f"select max(block_height) from public.validator_block;")
print(cur.fetchall())

[(9291903,)]


In [19]:
print("total blocks:", ((9291903 - label_win_size - combine_win_size) - (7049472 + vote_proposed_win_size - 1)) / 150 + 1)

total blocks: 14181.546666666667


In [20]:
14181 * 39

553059